In [35]:
import alphashape
import shapely.geometry as geometry
from shapely.geometry import Polygon
from descartes import PolygonPatch
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import netCDF4
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cc3d
from shapely.geometry import shape
from shapely.ops import transform
import pyproj
from functools import partial

## Define some definitions

In [36]:
def ow_lag(date):
    # Read from forward
    f_fw = netCDF4.Dataset('../data/forward/new_%s.nc' %date, 'r')
    f_bw = netCDF4.Dataset('../data/backward/new_%s.nc' %date, 'r')
    
    f_ieig1 = f_fw.variables['ieig1']
    f_reig1 = f_fw.variables['reig1']
    b_ieig1 = f_bw.variables['ieig1']
    b_reig1 = f_bw.variables['reig1']
    
    ieig1 = f_ieig1[:,:,:] + b_ieig1[:,:,:]
    reig1 = f_reig1[:,:,:] + b_reig1[:,:,:]

    ow_lag = ieig1[:,:,:] - b_ieig1[:,:,:]
    # Flush
    f_fw = None; f_bw = None; f_ieig1 = None; f_reig1 = None; b_ieig1 = None; b_reig1 = None
    ieig1 = None; reig1 = None
    
    return ow_lag

In [37]:
proj = partial(pyproj.transform, pyproj.Proj(init='epsg:4326'),
               pyproj.Proj(init='epsg:3857'))

In [38]:
date = "20170729_06"

In [39]:
df = pd.read_csv("../CLU_VORT/labels/labellist_%s.csv" %date, header = None, names = ["values","npoints","labels"])

In [40]:
df

,values,npoints,labels
0,0.0,11,1
1,0.0,11,2
2,0.0,11,3
3,0.0,11,12
4,0.0,11,21
...,...,...,...
1812,41.9,1,2
1813,42.0,1,2
1814,42.1,1,2
1815,42.2,1,2


In [41]:
npoints = df.groupby(["values"])["npoints"].count().to_numpy()

In [42]:
values = df.groupby(["values"]).count().index.to_numpy()

In [43]:
# Check 
val = values[10]
label = df[df["values"] == val]["labels"].to_numpy()

In [44]:
label

array([ 1,  2,  3,  4,  5,  6,  8, 14, 23, 24, 26, 29, 34, 39, 41, 43, 46,
       48, 50])

In [45]:
# Check file 1
f = netCDF4.Dataset("../CLU_VORT/nc_files/cc3d_%s.nc" %date,"r")

In [46]:
lons = f.variables['longitude'][:]
lats = f.variables['latitude'][:]
levs = f.variables['level'][:]

nlev = levs.shape[0]
nlon = lons.shape[0]
nlat = lats.shape[0]
print(nlon,nlat,nlev)

221 121 11


In [47]:
# Get OW_LAG
ow_lag = ow_lag(date)

In [48]:
# Sample var
ilev = 1
var = f.variables["labels_cc3d_%05.2f" %val][ilev,:,:]



In [51]:

for ilabel in label[2:3]:
    a_check = (var == ilabel).sum()
    
    lab_arr = (var == ilabel).astype('int')
    
#     print(lab_arr)
    labels_out = cc3d.connected_components(lab_arr,connectivity = 6)

    print(np.max(labels_out))

    for j in range(1,np.max(labels_out) + 1):
        ilat,ilon = np.where(labels_out == j)
        
        SMALL_THRESHOLD = 6
        count = len(ilat)
        if count > SMALL_THRESHOLD:
            latind_1 = lats[ilat]
            lonind_1 = lons[ilon]
         
            points = np.zeros([len(latind_1),2])
            for ipt in range(len(latind_1)):
                points[ipt,0] = lonind_1[ipt]
                points[ipt,1] = latind_1[ipt]
        
            alpha = 0.95 * alphashape.optimizealpha(points, max_iterations=100)
    
            hull = alphashape.alphashape(points, alpha)
            hull_lons,hull_lats = hull.exterior.coords.xy
            hull_ilon = np.searchsorted(lons,hull_lons)
            hull_ilat = np.searchsorted(lats,hull_lats)
            
            # Output centroids and area
            hull_centx = hull.centroid.x
            hull_centy = hull.centroid.y
#             hull_area = hull.area # in square degrees

            s = shape(hull)
            hull_area = transform(proj, s).area
            print(hull_area)
            #

            min_owlag = np.nanmin([x for x in ow_lag[ilev,hull_ilat,hull_ilon] if x != 0])
            
            print('{:>5.3f}\t{:>5d}\t{:<5d}\t{:>7.3f}\t{:>7.3f}\t{:>20.3f}\t{:e}'
                  .format(val,ilabel,j,hull_centx,hull_centy,hull_area,min_owlag),
                   file = open(r"cluster_%s.dat" %date, "a+"))
            
            
            # Start storing to file
            
            
            
        # Flush
        latind_1 = None; lonind_1 = None; ilat = None; ilon = None
        points = None; hull = None; hull_pts = None; alpha = None;
        hull_lons = None; hull_lats = None; hull_ilon = None; hull_ilat = None
        
    # Flush
    lab_arr = None; labels_out = None

6


264814869494.48502


381305831803.73114


# 

In [54]:
a,b,c = 3*[None]

In [57]:
print(b)

None
